In [ ]:
#data notes
'''
-software records without doi and storage dates were removed
'''

In [ ]:
import csv
import community
import pandas as pd
import networkx as nx
import math
import matplotlib.pyplot as plt
import pickle
import requests
import sys
import codecs
from bs4 import BeautifulSoup
import re
import os

## Retrieve Project entries from OpenAire

We use Beautiful Soup library to parse records, retrieved from OpenAire.
Each request results in a soup object, which then can be parsed.

In [ ]:
#get a list of project soups from open aire and write a soupfile for each retrieved soup
#projects starting url: 'http://api.openaire.eu/oai_pmh?verb=ListRecords&metadataPrefix=oaf&set=ECProjects'
def writeSoupFilesNoLimit(url, outputType):
    session = requests.session()
    session.keep_alive = False
    directory = './' + outputType + 'soups/'
    #for continuing from the last resumption token use url below
    #url = 'http://api.openaire.eu/oai_pmh?verb=ListRecords&resumptionToken=272470|oaf|*%20AND%20(resulttypeid%20exact%20"publication"%20and%20funder%20exact%20"EC")|101000|5beec22045e3121cdfbc5abf|false|ECPublications'
    
    #get resumption token for the next request from the previous soup
    obtainedtoken = getIterationToken(url,session, directory)
    #print obtainedtoken
    strmaxpage = obtainedtoken.split("|")[0]
    #print strmaxpage
    maxpage = int(strmaxpage)//1000 + 1
    #print maxpage
    for i in range(0, maxpage):
        url = 'http://api.openaire.eu/oai_pmh?verb=ListRecords&resumptionToken=' + obtainedtoken
        obtainedtoken = getIterationToken(url, session, directory)
        print obtainedtoken

In [ ]:
#transform resumption token into appropriate string: https://www.openarchives.org/OAI/openarchivesprotocol.html#HTTPRequestFormat
def getGoodToken(totalstring):
    result = totalstring.replace(' ', '%20')
    return result;

In [ ]:
#takes a request url, retrieves a soup, saves a file for that soup, returns resumption token
def getIterationToken(url, session, directory):
    resp = session.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    nexttoken = soup.find(re.compile("^oai:resumptiontoken"))
    if nexttoken: #if nexttoken is not equal to 'None'
        filename = directory + 'soup' + str(int(nexttoken['cursor']) + 1000) + '.txt'
        with open(filename, 'w') as f: 
            f.write(str(soup))
        return getGoodToken(nexttoken.text);
    else:
        filename = directory + 'souplast' + '.txt'
        with open(filename, 'w') as f: 
            f.write(str(soup))

In [ ]:
#this endpoint url is obtained from OpenAire API webpage
writeSoupFilesNoLimit('http://api.openaire.eu/oai_pmh?verb=ListRecords&metadataPrefix=oaf&set=ECProjects', 'projects')

## Retrieve Publication entries from OpenAire

In [ ]:
#this endpoint url is obtained from OpenAire API webpage
writeSoupFilesNoLimit('http://api.openaire.eu/oai_pmh?verb=ListRecords&metadataPrefix=oaf&set=ECPublications', 'publications')

## Retrieve Software entries from OpenAire

In [ ]:
#get a list of software soups from open aire and write a soupfile for each retrieved soup
#WORKS IF THE NUMBER OF RECORDS IS < 10000
def writeSoupFiles(outputType):
    soups = list()
    maxpages = 100
    pagesize = 500 #1000 causes memory error for jupyter
    i = 1
    #for i in range(1, maxpages+1):
    while i < maxpages+1:
        requestring = 'http://api.openaire.eu/search/' + outputType + '?page=' + str(i) + '&size=' + str(pagesize) + '&hasECFunding=true'
        print requestring
        resploop = requests.get(requestring)
        souploop = BeautifulSoup(resploop.content, 'lxml')
        filename = './' + outputType + 'soups/' + 'soup' + str(i) + '.txt'
        if i==1:# just for the first iteration, as we want to get the total number of records
            total = souploop.find('total')
            maxpages = int(total.text)//pagesize + 1
        print i,"/", maxpages
        with open(filename, 'w') as f: 
            f.write(str(souploop))
        i+=1
        #soups.append(souploop)
    #return soups;

In [ ]:
writeSoupFiles('software')

In [ ]:
#test cell
'''resp = requests.get('http://api.openaire.eu/oai_pmh?verb=ListRecords&resumptionToken=46499|oaf|*%20AND%20(oaftype%20exact%20"project"%20and%20funder%20exact%20"EC")|46000|5c0a0349dc92c50c38b1ec20|false|ECProjects')
soup = BeautifulSoup(resp.content, 'lxml')
nexttoken = soup.find(re.compile("^oai:resumptiontoken"))
print nexttoken
directory = './' + 'projects' + 'soups/'
if nexttoken: #if nexttoken is not none
    print 'not last'
else:
    print 'last'
    filename = directory + 'souplast' + '.txt'
    with open(filename, 'w') as f: 
        f.write(str(soup))'''